In [127]:
# %pip install requests 

In [128]:
import requests
from bs4 import BeautifulSoup

def content_scraper(soup, links, identifier):
    link_and_data = {}

    for link in links:  # Iterate over all links
        response = requests.get(link)  # Use the current link

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Handle scraping logic based on the identifier
            match identifier:
                case 'jena':
                    title = soup.find_all('title')
                    content = soup.find_all('div', class_='flex-layout')

                    # Extract title and content if found
                    text_content = (
                        title[0].get_text(strip=True) + ': ' + content[0].get_text(strip=True)
                        if title and content else "No content found"
                    )
                case 'siegburg':
                    divs = soup.find_all('div', class_='flex-layout')
                    content = divs[1] if len(divs) > 1 else None

                    # Extract content if found
                    text_content = content.get_text(strip=True) if content else "No content found"
                case _:
                    text_content = "Wrong identifier"

            link_and_data[link] = text_content
        else:
            # Handle failed requests
            link_and_data[link] = f"Failed to retrieve the webpage. Status code: {response.status_code}"

    return link_and_data


In [129]:
def siegburg_data(soup):
    links = []

    a_tags = soup.find_all('a')

    for tag in a_tags:
        href = tag.get('href')
        
        if href and '/proposals/' in href and not 'new' in href:
            link = 'https://mitmachen.siegburg.de' + href
    
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'siegburg')

    return link_and_content


def jena_data(soup):
    links = []

    a_tags = soup.find_all('a',  class_='resource-item--title')

    for tag in a_tags:
        href = tag.get('href')
        
        if href:
            link = 'https://mitmachen.jena.de' + href
            
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'jena')

    return link_and_content

In [130]:
urls = [
        'https://mitmachen.siegburg.de/angebotslandkarte',
        'https://mitmachen.jena.de/projekts'
    ]


def def_42(urls):
    urls_and_data = {}

    for url in urls:
        response = requests.get(url)
        if response.status_code != 200:
            print("Failed to retrieve the webpage. Status code:", response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        link_and_content = scraper(soup, url)
        urls_and_data[url] = link_and_content

    return urls_and_data


def scraper(soup, url):
    if 'siegburg' in url:
        return siegburg_data(soup)
    elif 'jena' in url:
        return jena_data(soup)
    
    return "No scraper defined for this URL"


link_and_data = def_42(urls)

In [131]:
link_and_data

{'https://mitmachen.siegburg.de/angebotslandkarte': {'https://mitmachen.siegburg.de/proposals/1090-senioiren-cafe-lichtblick-kaldauen': 'Senioiren Cafe Lichtblick Kaldauen18. September 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte60-7475+Geselliges BeisammenseinDas Cafe Lichtblick in Kaldauen lädt alle Senioren zum geselligen Beisammensein bei Kaffee und Kuchen ein. Es wird gesungen, gespielt, getanzt, gelacht und viel erzählt. Auch gemeinsame Ausflüge und Feiern stehen auf dem Programm.Haben Sie Interesse, dann kommen Sie gerne vorbei. Die Senioren und das Team freut sich auf Sie!Treffen : Alle 14 Tage Dienstags von 15.00 - 17.00 Uhr\xa0(immer in geraden Wochen)Ort : Pfarrheim der katholischen Kirche KaldauenAntoniusweg 1 ;\xa053721 SiegburgAnmeldungen bei Rita Quadt ; Tel.:\xa0015733725174:Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.MCafé LichtblickBeigetreten am: 18.09.2024Konto verifiziertVorschläge1Unterstützer*innenUnterstützer*inne

In [132]:
import pandas as pd

# Separate URLs and content for Siegburg and Jena
siegburg_data = {k: v for k, v in link_and_data.items() if "siegburg" in k}
jena_data = {k: v for k, v in link_and_data.items() if "jena" in k}

# Create DataFrames
df_sieburg = pd.DataFrame(list(siegburg_data.items()), columns=["URL", "Content"])
df_jena = pd.DataFrame(list(jena_data.items()), columns=["URL", "Content"])


In [133]:
df_jena = pd.DataFrame(df_jena['Content'][0].items(), columns=["URL", "Content"])

In [134]:
df_sieburg = pd.DataFrame(df_sieburg['Content'][0].items(), columns=["URL", "Content"])

In [135]:
import re

# Enhanced function to extract all logical parts, including "Unterstützer*innen"
def extract_full_data_with_supporters(content):
    # Extract title (everything before the first date)
    title_match = re.search(r'^(.*?)(\r|\d{1,2}\.\s\w+\s\d{4})', content)
    title = title_match.group(1).strip() if title_match else None

    # Extract date
    date_match = re.search(r'\d{1,2}\.\s\w+\s\d{4}', content)
    date = date_match.group(0) if date_match else None

    # Extract comments count
    comments_match = re.search(r'(\d+)\sKommentare', content)
    comments = int(comments_match.group(1)) if comments_match else 0

    # Extract tags (sections with numbers or + signs)
    tags_match = re.findall(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
    tags = ', '.join(tags_match) if tags_match else None

    # Extract description (everything after "Geselliges Beisammensein" or similar patterns)
    description_start = re.search(r'(Geselliges Beisammensein|Angebotslandkarte)', content)
    description = content[description_start.start():].strip() if description_start else None

    # Extract username
    username_match = re.search(r'(\w+\s\w+|Beigetreten am:.*?\d{4})', content)
    username = username_match.group(1).split('Beigetreten am:')[0].strip() if username_match else None

    # Extract Vorschläge count
    vorschlaege_match = re.search(r'Vorschläge(\d+)', content)
    vorschlaege = int(vorschlaege_match.group(1)) if vorschlaege_match else 0

    # Extract Konto verification status
    konto_match = re.search(r'(Konto\s(verifiziert|ist nicht verifiziert))', content)
    konto_status = konto_match.group(2) if konto_match else None

    # # Extract registration date
    # registration_match = re.search(r'Beigetreten am:\s(\d{1,2}\.\s\w+\s\d{4})', content)
    # registration_date = registration_match.group(1) if registration_match else None

    # Extract number of Unterstützer*innen
    supporters_match = re.search(r'(\d+)\sUnterstützer\*in', content)
    supporters = int(supporters_match.group(1)) if supporters_match else 0

    return title, date, comments, tags, description, username, vorschlaege, konto_status, supporters

# Apply the enhanced function to the DataFrame and create new columns
df_sieburg[['Title', 'Date', 'Comments', 'Tags', 'Description', 'Username', 'Vorschläge', 'Konto Status', 'Supporters']] = df_sieburg['Content'].apply(
    lambda x: pd.Series(extract_full_data_with_supporters(x))
)



In [137]:
df_jena

,URL,Content
0,https://mitmachen.jena.de/matschspielplatz-sch...,Matschspielplatz Schlippenstraße: In Wenigenje...
1,https://mitmachen.jena.de/buergerbudget,Bürgerbudget 2024: Ergebnis Bürgerbudget Jena ...
2,https://mitmachen.jena.de/rahmenplanung-westba...,Rahmenplanung Westbahnhof und Umfeld: Anlass d...
3,https://mitmachen.jena.de/alteschwimmhalle,Nachnutzung der alten Schwimmhalle in Lobeda-W...
4,https://mitmachen.jena.de/jenaliebe,Wie liebst Du unsere Stadt?: #JenaLiebe – Befr...
5,https://mitmachen.jena.de/klimagerechte-platzg...,Klimagerechte Platzgestaltung Ernst-Abbe-Platz...
6,https://mitmachen.jena.de/buergerbudget-2023,Bürgerbudget 2023: Ergebnis Bürgerbudget Jena ...
7,https://mitmachen.jena.de/smartcity,"Smart City: Jena digitalisiert, lernt und teil..."
8,https://mitmachen.jena.de/buergerbudget-2022,Bürgerbudget 2022: Ergebnis Bügerbudget Jena 2...
9,https://mitmachen.jena.de/stufe-1-kurzfristige...,Stufe I - Kurzfristige Entwickelbarkeit: Der S...


In [138]:
# Generalized function to process all entries in df_jena['Content']
def process_all_jena_entries(df_jena):
    # Generalized function for extracting data
    def extract_jena_data(content):
        # Extract Title
        title_match = re.search(r'^(.*?):', content)
        title = title_match.group(1).strip() if title_match else None

        # Extract Description
        description_match = re.search(r':\s*(.*?)\n⭐', content, re.DOTALL)
        description = description_match.group(1).strip() if description_match else None

        # Extract Project Start Date
        start_date_match = re.search(r'Projektstart\s*(\d{1,2}\.\s\w+\s\d{4})', content)
        start_date = start_date_match.group(1) if start_date_match else None

        # Extract Username
        username_match = re.search(r'@(\w+)', content)
        username = username_match.group(1) if username_match else None

        # Extract Tags
        tags_match = re.findall(r'#(\w+)', content)
        tags = ', '.join(tags_match) if tags_match else None

        # Extract Supporters Count
        supporters_match = re.search(r'(\d+)\sUnterstützer\*in', content)
        supporters = int(supporters_match.group(1)) if supporters_match else None

        # Extract Number of Discussions
        discussions_match = re.search(r'DiskussionenAbgeschlossen\sam\s(\d{1,2}\.\s\w+\s\d{4})', content)
        discussions = discussions_match.group(1) if discussions_match else None

        # Extract Comments (text, username, date, likes/dislikes)
        comments = []
        comment_matches = re.finditer(r'Kommentar:(.*?)@(\w+)\s\((\d{1,2}\.\s\w+\s\d{4})\).*?Likes:(\d+).*?Dislikes:(\d+)', content, re.DOTALL)
        for match in comment_matches:
            comment_text = match.group(1).strip()
            comment_user = match.group(2).strip()
            comment_date = match.group(3).strip()
            likes = int(match.group(4))
            dislikes = int(match.group(5))
            comments.append({
                'Text': comment_text,
                'Username': comment_user,
                'Date': comment_date,
                'Likes': likes,
                'Dislikes': dislikes
            })

        return {
            "Title": title,
            "Description": description,
            "Project Start Date": start_date,
            "Username": username,
            "Tags": tags,
            "Supporters": supporters,
            "Discussions": discussions,
            "Comments": comments
        }

    # Apply the extraction function to all rows in df_jena['Content']
    extracted_data = df_jena['Content'].apply(extract_jena_data)

    # Create a main DataFrame for structured data
    df_jena_cleaned = pd.DataFrame([{
        "URL": row["URL"],
        "Title": data["Title"],
        "Description": data["Description"],
        "Project Start Date": data["Project Start Date"],
        "Username": data["Username"],
        "Tags": data["Tags"],
        "Supporters": data["Supporters"],
        "Discussions": data["Discussions"]
    } for row, data in zip(df_jena.to_dict('records'), extracted_data)])

    # Create a separate DataFrame for comments
    comments_list = []
    for url, data in zip(df_jena["URL"], extracted_data):
        for comment in data["Comments"]:
            comment["Project URL"] = url  # Link comment to project URL
            comments_list.append(comment)

    df_comments = pd.DataFrame(comments_list)

    return df_jena_cleaned, df_comments


# Example Usage (replace this with actual df_jena)
df_jena_cleaned, df_comments = process_all_jena_entries(df_jena)


In [139]:
df_jena_cleaned

,URL,Title,Description,Project Start Date,Username,Tags,Supporters,Discussions
0,https://mitmachen.jena.de/matschspielplatz-sch...,Matschspielplatz Schlippenstraße,None,None,None,None,None,None
1,https://mitmachen.jena.de/buergerbudget,Bürgerbudget 2024,None,28. Mai 2024,jena,None,None,None
2,https://mitmachen.jena.de/rahmenplanung-westba...,Rahmenplanung Westbahnhof und Umfeld,None,None,None,None,None,None
3,https://mitmachen.jena.de/alteschwimmhalle,Nachnutzung der alten Schwimmhalle in Lobeda-West,None,None,None,None,None,None
4,https://mitmachen.jena.de/jenaliebe,Wie liebst Du unsere Stadt?,None,None,None,"JenaLiebe, JenaLiebe",None,None
5,https://mitmachen.jena.de/klimagerechte-platzg...,Klimagerechte Platzgestaltung Ernst-Abbe-Platz...,None,18. März 2024,None,None,None,None
6,https://mitmachen.jena.de/buergerbudget-2023,Bürgerbudget 2023,None,25. Mai 2023,jena,None,None,None
7,https://mitmachen.jena.de/smartcity,Smart City,None,16. März 2022,Jugend,Jugend,None,02. Juli 2022
8,https://mitmachen.jena.de/buergerbudget-2022,Bürgerbudget 2022,None,27. Mai 2022,jena,None,None,None
9,https://mitmachen.jena.de/stufe-1-kurzfristige...,Stufe I - Kurzfristige Entwickelbarkeit,None,04. November 2024,None,None,None,None


In [160]:
import requests
from bs4 import BeautifulSoup
import re

# Updated function to extract comments from a single page
def extract_comments_from_page(soup):
    comments_data = []
    comments_section = soup.find_all('div', class_='comment small-12')
    
    for comment in comments_section:
        # Extract comment text
        comment_text = comment.find('p').get_text(strip=True) if comment.find('p') else None
        
        # Extract username
        username_tag = comment.find('span', class_='user-name')
        username = username_tag.get_text(strip=True) if username_tag else None
        
        # Extract date
        date_tag = comment.find('div', class_='comment-info').find_all('a')[-1]
        date = date_tag.get_text(strip=True) if date_tag else None

        
        # Extract likes and dislikes (clean and convert to integer)
        likes_tag = comment.find('span', class_='in-favor')
        likes = int(re.sub(r'\D', '', likes_tag.get_text(strip=True))) if likes_tag else 0
        
        dislikes_tag = comment.find('span', class_='against')
        dislikes = int(re.sub(r'\D', '', dislikes_tag.get_text(strip=True))) if dislikes_tag else 0
        
        # Extract total votes (clean and convert to integer)
        total_votes = likes + dislikes
        
        comments_data.append({
            'Text': comment_text,
            'Username': username,
            'Date': date,
            'Likes': likes,
            'Dislikes': dislikes,
            'Total Votes': total_votes
        })
    return comments_data


# Scrape all comments across pages (pagination logic remains the same)
def scrape_all_comments(base_url):
    comments = []
    page = 1
    
    while True:
        paginated_url = f"{base_url}?page={page}" if page > 1 else base_url
        response = requests.get(paginated_url)
        
        if response.status_code != 200:
            print(f"Failed to load page {page} for URL: {base_url}")
            break
        
        soup = BeautifulSoup(response.content, 'html.parser')
        comments_on_page = extract_comments_from_page(soup)
        
        if not comments_on_page:  # Stop if no comments on the page
            break
        
        comments.extend(comments_on_page)
        page += 1

    return comments

# Function to scrape the main content and comments for each URL
def scrape_content_and_comments(urls):
    data = []
    
    for url in urls:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to load URL: {url}")
            continue
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Scrape main content
        title = soup.find('title').get_text(strip=True) if soup.find('title') else None
        content_div = soup.find('div', class_='flex-layout')
        content = content_div.get_text(strip=True) if content_div else None
        
        # Scrape comments
        comments = scrape_all_comments(url)
        
        data.append({
            'URL': url,
            'Title': title,
            'Content': content,
            'Comments': comments
        })
    
    return data

# Scrape comments for all URLs
urls = df_jena['URL'].tolist()
scraped_data = scrape_content_and_comments(urls)

# Create structured DataFrame for comments
comments_data = []
for item in scraped_data:
    for comment in item['Comments']:
        comment['URL'] = item['URL']  # Link comment to the project URL
        comments_data.append(comment)

# Create the comments DataFrame
df_comments = pd.DataFrame(comments_data)

# Create a mapping from URL to Title
url_to_title = df_jena_cleaned.set_index('URL')['Title'].to_dict()

# Add a 'Project' column to df_comments using the mapping
df_comments['Project'] = df_comments['URL'].map(url_to_title)
df_comments = df_comments[['URL', 'Project'] + [col for col in df_comments.columns if col not in ['URL', 'Project']]]

In [164]:
pd.set_option('display.max_colwidth', 100)

In [165]:
df_comments

,URL,Project,Text,Username,Date,Likes,Dislikes,Total Votes
0,https://mitmachen.jena.de/stufe-1-kurzfristige-entwickelbarkeit,Stufe I - Kurzfristige Entwickelbarkeit,"Ich kann nicht ganz nachvollziehen, warum in der Planung immer wieder ein Bedeutungsverlust für ...",r_luen,05. November 2024 17:16:56,9,0,9
1,https://mitmachen.jena.de/stufe-1-kurzfristige-entwickelbarkeit,Stufe I - Kurzfristige Entwickelbarkeit,"Den Campusradweg kann man ruhig ins Reich der Fantasie verbannen, verläuft doch dieser auf Bahna...",Klaus.Kleiner,13. November 2024 05:29:18,6,2,8
2,https://mitmachen.jena.de/stufe-1-kurzfristige-entwickelbarkeit,Stufe I - Kurzfristige Entwickelbarkeit,Stichwort: Straßenbahn!,Klaus.kleiner.77,17. November 2024 09:25:14,2,1,3
3,https://mitmachen.jena.de/stufe-1-kurzfristige-entwickelbarkeit,Stufe I - Kurzfristige Entwickelbarkeit,"Bis auf I-B und I-D ist das mehr oder weniger nur Aufhübschen der aktuellen Situation, die an de...",Brabax,08. November 2024 20:08:28,6,0,6
4,https://mitmachen.jena.de/stufe-1-kurzfristige-entwickelbarkeit,Stufe I - Kurzfristige Entwickelbarkeit,Eine Straßenbahn die völlig unrealistisch ist! Braucht man doch für die Planung für eine Verläng...,Klaus.Kleiner,13. November 2024 05:31:55,5,4,9
...,...,...,...,...,...,...,...,...
309,https://mitmachen.jena.de/szenario-3-langfristige-flaechenverfuegbarkeit-2,Szenario 3 „langfristige Flächenverfügbarkeit“,"An dem Entwurf gefällt mir im Gegensatz zu den anderen, dass das Gebäude auf der Westseite nicht...",Schlo,05. Februar 2024 22:19:32,0,2,2
310,https://mitmachen.jena.de/szenario-3-langfristige-flaechenverfuegbarkeit-2,Szenario 3 „langfristige Flächenverfügbarkeit“,Die B&R unter dem Empfangsgebäude entdeckt?,Touringer,05. Februar 2024 22:45:49,0,0,0
311,https://mitmachen.jena.de/szenario-3-langfristige-flaechenverfuegbarkeit-2,Szenario 3 „langfristige Flächenverfügbarkeit“,"Szenario 3 hat zwar den Vorteil, die Verknüpfung zw. ÖPNV und Bahn zu optimieren. Wenn langfrist...",KarSteN,06. Februar 2024 23:32:51,1,4,5
312,https://mitmachen.jena.de/szenario-3-langfristige-flaechenverfuegbarkeit-2,Szenario 3 „langfristige Flächenverfügbarkeit“,Auch die Aufwertung des Bahnhofs kann keine 15-20 Jahre warten. Insofern sollte hinterfragt werd...,"Bündnis ""Fernverkehr für Jena""",16. Februar 2024 13:51:18,2,0,2


In [143]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import json

# Initialize the T5 model and tokenizer
model_name = "t5-small"  # You can also try "t5-base" or "t5-large" for better performance
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Example raw data
raw_data = df_jena["Content"][0]

# Create a prompt for structuring the text
prompt = f"""Structure the following text into a JSON format with fields URL, Title, Date, Description, Tags, Comments, Votes, Participants, Contact, Related Links:

{raw_data}
"""

# Tokenize input
inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

# Generate structured output
outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
structured_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Attempt to parse the output as JSON
try:
    # If the model output resembles JSON, use json.loads
    structured_data = json.loads(structured_text)
    # Convert the structured data into a DataFrame
    df_jena_structured = pd.DataFrame([structured_data])
    print(df_jena_structured)  # Display the structured DataFrame
except json.JSONDecodeError:
    # Print raw output and handle malformed data
    print("Model output is not valid JSON. Here's the raw output:")
    print(structured_text)


Model output is not valid JSON. Here's the raw output:
,, Date, Description, Tags, Votes, Participants, Contact, Related Links: Matschspielplatz Schlippenstraße: In Wenigenjena, konkret auf dem Spielplatz Schlippenstraße, soll der erste Matschspielplatz entstehen. In dieser Phase können Kinder und Sorgeberechtigte ihre Ideen und Wünsche auf analogen und digitalen Wegen einbringen.PlanungsphaseAb Januar 2025 wird die Aufgabe der Spielplatzplanung an ein Planungsbüro vergeben. Diese führt gemeinsam mit der Stadtverwal


In [144]:
# df['Comments'].replace(20240, 0, inplace=True)

In [145]:
# Function to clean description considering keywords, numeric patterns, and refined starting logic
def clean_description_advanced(content):
    # Define keywords that mark the beginning of the description
    keywords = [
        'Geselliges Beisammensein', 'Natur', 'Hilfe & Beratung', 'Bildung', 
        'Musik', 'Bewegung', 'Glaube', 'Kulinarisches', 'Kunst & Kultur', 'Sonstiges',
    ]
    
    # Check for keywords first
    for keyword in keywords:
        if keyword in content:
            start_idx = content.find(keyword) + len(keyword)
            description = content[start_idx:].strip()
            description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
            return description

    # If no keyword is found, check for numeric patterns like "18-24, 25-49, etc."
    numeric_pattern = re.search(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
    if numeric_pattern:
        start_idx = numeric_pattern.end()
        description = content[start_idx:].strip()
        description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
        return description

    # As a fallback, find the first capital letter, quote, or digit to mark the start
    fallback_match = re.search(r'[A-Z"0-9]', content)
    if fallback_match:
        start_idx = fallback_match.start()
        description = content[start_idx:].strip()
        description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
        return description

    # If nothing works, return the content as is
    return content

# Apply the advanced cleaning function to the Description column
df['Description'] = df['Content'].apply(clean_description_advanced)


NameError: name 'df' is not defined

In [ ]:
df

,URL,Content,Title,Date,Comments,Tags,Description,Username,Vorschläge,Konto Status,Supporters
0,https://mitmachen.siegburg.de/proposals/1090-s...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,0,"60-74, 75+",Das Cafe Lichtblick in Kaldauen lädt alle Seni...,Senioiren Cafe,1,verifiziert,0
1,https://mitmachen.siegburg.de/proposals/1089-d...,Digitalsprechstunde für Senior*innen16. Septem...,Digitalsprechstunde für Senior*innen,16. September 2024,0,"60-74, 75+",Am ersten Donnerstag im Monat erhalten Seniori...,Digitalsprechstunde für,17,verifiziert,0
2,https://mitmachen.siegburg.de/proposals/1087-g...,Gemeinschaftsgarten Cecilienstraße12. Septembe...,Gemeinschaftsgarten Cecilienstraße,12. September 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+, ...",Der Gemeinschaftsgarten ist eine Kooperation d...,Gemeinschaftsgarten Cecilienstraße12,1,ist nicht verifiziert,0
3,https://mitmachen.siegburg.de/proposals/1056-g...,Geselliges Beisammensein in Braschoß18. August...,Geselliges Beisammensein in Braschoß,18. August 2024,0,"18-24, 25-49, 50-59, 60-74, 75+",in Braschoß18. August 20240 KommentareZugehöri...,Geselliges Beisammensein,3,verifiziert,4
4,https://mitmachen.siegburg.de/proposals/1055-b...,Bildungsveranstaltungen Pfarrverein Braschoß18...,Bildungsveranstaltungen Pfarrverein Braschoß,18. August 2024,0,"18-24, 25-49, 50-59, 60-74, 75+",sveranstaltungen Pfarrverein Braschoß18. Augus...,Bildungsveranstaltungen Pfarrverein,3,verifiziert,4
5,https://mitmachen.siegburg.de/proposals/1054-o...,Öffentlicher Bücherschrank für Kinder-/Jugend-...,Öffentlicher Bücherschrank für Kinder-/Jugend-...,15. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+",Stöbern und Tauschen erwünscht!Den Bücherschra...,Öffentlicher Bücherschrank,3,verifiziert,2
6,https://mitmachen.siegburg.de/proposals/1052-r...,Rikscha-Fahrten in Siegburg14. August 20240 Ko...,Rikscha-Fahrten in Siegburg,14. August 2024,0,75+,Ausfahrten mit der Fahrrad Rikscha in die nähe...,Fahrten in,2,ist nicht verifiziert,4
7,https://mitmachen.siegburg.de/proposals/1051-m...,Mach was freiwillig!\r\n\r\nEhrenamt und Engag...,Mach was freiwillig!,12. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+, ...",Die Freiwilligen-Agentur für den Rhein-Sieg-Kr...,Mach was,1,ist nicht verifiziert,2
8,https://mitmachen.siegburg.de/proposals/1049-g...,Generationen-Parcours08. August 20240 Kommenta...,Generationen-Parcours,08. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+",Die Outdoor Fitness Anlage am Michaelsberg unt...,August 20240,17,verifiziert,0
9,https://mitmachen.siegburg.de/proposals/1041-e...,Erlebe Abenteuer mit den Annopfadfinder*innen!...,Erlebe Abenteuer mit den Annopfadfinder*innen!,17. Juli 2024,0,"0-12, 13-17, 18-24",Wir sind diePfadfinder*innen vom Stamm St. Ann...,Erlebe Abenteuer,1,verifiziert,1


In [ ]:
df.to_csv('siegburg_data.csv', index=False)

In [ ]:
df['URL'][14]

'https://mitmachen.siegburg.de/proposals/1030-sommerferien-workshop-self-defense-4-girls'